## Set Up

In [1]:
%run D:/KIMoDIs/global-groundwater-models-main/notebooks/plots_set_up.ipynb

In [2]:
# Load error metrics
metrics_subset = pq.read_table(os.path.join(RESULT_PATH, 'metrics', 'median_metrics_subset.parquet'))
metrics_subset = metrics_subset.to_pandas()

### Areas with higher model accuracy


In [3]:
# Median NSE per static feature category
static_subset = test_df_in_sample[['proj_id', 
                                   'aquifer_type',
                                   'hyraum_gr', 
                                   'land_cover', 
                                   'permeability_coef',
                                   'soil_texture', 
                                   'twi',
                                   'gw_recharge',
                                   'landform_sha10km',
                                   'eumohp_dsd1',
                                   'eumohp_lp1',
                                   'eumohp_sd1',
                                   'elevation']].drop_duplicates()

metrics_subset = metrics_subset.merge(static_subset, on='proj_id')
metrics_nse05 = metrics_subset[(metrics_subset['horizon']==12) & (metrics_subset['NSE']>=0.5)]

# loop for each static categorical variable and output tables
static_cat = ['aquifer_type', 'hyraum_gr', 'land_cover', 'permeability_coef', 'soil_texture']

for VAR in static_cat: 
    _df = pd.DataFrame(metrics_subset[metrics_subset['horizon']==12].groupby(['model_type', VAR])['NSE'].median())
    _df = pd.merge(_df, pd.DataFrame(metrics_subset[metrics_subset['horizon']==12].groupby(['model_type', VAR]).size().reset_index()), 
         on=['model_type', VAR])
    _df = _df.rename(columns = {0:'N'})
    _df = _df.round({'NSE':2})
    _df = _df.pivot(columns='model_type', index=[VAR, 'N'])
    _df[[('NSE', 'tft_full'), ('NSE', 'tft_dyn'), ('NSE', 'nhits_full'), ('NSE', 'nhits_dyn')]].to_csv(os.path.join(SHARE_PATH, 
                                                                           'global_mod_paper', 
                                                                           'results', 
                                                                           f'nse_per_{VAR}.csv'))

### Correlation NSE with numeric static feature values

In [4]:
# Helper function to calculate and create correlation output
def create_corr_tbl(data: pd.DataFrame, FEATURE):
    corr_df = data.groupby('model_type').apply(
                            lambda group: pd.Series(spearmanr(group['NSE'], group[FEATURE]), 
                                                    index=['corr', 'p_val'])
                        ).reset_index()  
    return corr_df

#     corr_df = data.groupby('model_type')[['NSE', TS_FEATURE]].corr(method='spearman').iloc[0::2].reset_index()
#     corr_df = corr_df.pivot(columns='model_type', index=['level_1', 'NSE'])
#     corr_df = corr_df[[(TS_FEATURE, 'tft_full'), (TS_FEATURE, 'tft_dyn'), (TS_FEATURE, 'nhits_full'), (TS_FEATURE, 'nhits_dyn')]]

In [41]:
static_num = ['twi', 'gw_recharge', 'landform_sha10km', 'eumohp_dsd1', 'eumohp_lp1', 'eumohp_sd1', 'elevation']

corr_res = []
for VAR in static_num: 
    
    # Correlation coef (spearman)
    _corr = create_corr_tbl(metrics_subset[metrics_subset['horizon']==12], FEATURE=VAR)
    _corr['feature'] = VAR    
    corr_res.append(_corr)

# Combine all the correlation tables into a single DataFrame
corr_df = pd.concat(corr_res, ignore_index=True)
corr_df = corr_df.pivot(columns='model_type',  index=['feature'])
corr_df = corr_df[[('corr', 'tft_full'), ('corr', 'tft_dyn'), ('corr', 'nhits_full'), ('corr', 'nhits_dyn'),
                   ('p_val', 'tft_full'), ('p_val', 'tft_dyn'), ('p_val', 'nhits_full'), ('p_val', 'nhits_dyn')]]

corr_df.to_excel(os.path.join(SHARE_PATH, 'global_mod_paper', 'results', 'nse_static_num_spearman.xlsx')) 

### Correlation with time series features

- Seasonal behaviour
- Length of training period
- Flashiness/SDdiff
- Amplitude 
- Boundness
- Seasonality‐magnitude
- Linearer Trend (für definierte Zeit) 

In [7]:
# Use the complete time series of each monitoring well
compl_df = pd.concat([train_df[['proj_id', 'time', 'time_idx', 'gwl']], 
                      val_df[['proj_id', 'time', 'time_idx', 'gwl']], 
                      test_df_in_sample[['proj_id', 'time', 'time_idx', 'gwl']]
                      ]
                     )
compl_df = compl_df.sort_values(by=['proj_id', 'time'])

In [31]:
create_corr_tbl(metrics_subset[metrics_subset['horizon']==12], FEATURE='twi')

,model_type,corr,p_val
0,nhits_dyn,0.243893,0.0
1,nhits_full,0.218280,0.0
2,tft_dyn,0.177413,0.0
3,tft_full,0.177255,0.0


#### Seasonal behaviour

"Position of the maximum in the annual cycle, agreement with the expected average seasonality (Min in Sep, Max in March)".
Correlation tells if the models perform better when time series follow the expected seasonality

- Calculated according to Wunsch et al. 2022 b:
    - Mean per month
    - Correlate (pearson) with sinus wave following the typical gwl seasonality 
    - Obtain metric by dividing the correlation with the amplitude (diff gwl and sinus wave) 
    - Then correlate NSE with the metric 


In [14]:
compl_df.loc[:,'year'] = compl_df['time'].dt.year
compl_df.loc[:,'month'] = compl_df['time'].dt.month

# Z transform GWL
compl_df.loc[:,'z_gwl'] = compl_df.groupby('proj_id')['gwl'].transform(lambda x: (x - x.mean())/x.std())

from utils import seasonal_behaviour

sb_df = seasonal_behaviour(data=compl_df[['proj_id', 'time', 'month', 'z_gwl']])
metrics_subset = metrics_subset.merge(sb_df[['proj_id', 'sb_metric']], on='proj_id')
sb_nse_corr = metrics_subset[metrics_subset['horizon']==12][['proj_id', 'model_type', 'horizon', 'NSE', 'sb_metric']].drop_duplicates()
# sb_nse_corr.groupby('model_type')[['NSE', 'sb_metric']].corr(method='spearman').iloc[0::2]

In [8]:
sb_nse_corr = create_corr_tbl(data=sb_nse_corr, TS_FEATURE='sb_metric')
sb_nse_corr.to_excel(os.path.join(SHARE_PATH, 'global_mod_paper', 'results', 'nse_seas_beh_spearman.xlsx'))
sb_nse_corr

,corr,p_val
model_type,,
nhits_dyn,0.342234,3.295548e-145
nhits_full,0.283058,5.329854e-98
tft_dyn,0.304912,3.530492e-114
tft_full,0.267135,4.280671e-87


#### Length of training period

In [11]:
ts_length = pd.DataFrame(compl_df['proj_id'].value_counts()).reset_index()
metrics_subset = metrics_subset.merge(ts_length, on = 'proj_id')

In [15]:
ts_nse_corr = create_corr_tbl(data=metrics_subset[metrics_subset['horizon']==12], TS_FEATURE='count')
ts_nse_corr.to_excel(os.path.join(SHARE_PATH, 'global_mod_paper', 'results', 'nse_tslength_spearman.xlsx'))
ts_nse_corr

,model_type,corr,p_val
0,nhits_dyn,0.027762,2.658001e-12
1,nhits_full,0.018807,2.160486e-06
2,tft_dyn,-0.002237,5.731261e-01
3,tft_full,0.009750,1.404980e-02


#### Flashiness/SDdiff

"The feature SDdiff describes how often strong rates of changes within a time series occur. It is therefore a measure of flashiness and variability. We use the standard deviation σ of the first derivative of the original, unscaled, unnormalized time series date for calculation."
The correlation tells us if our models perform worse on time series with sudden changes. 

In [5]:
# from utils import SD_diff

def SD_diff(x):
    # Compute the differences between consecutive elements
    differences = np.diff(x)
    # Compute the standard deviation
    std_dev = np.nanstd(differences)
    return std_dev

In [8]:
sddiff_df = pd.DataFrame(compl_df.groupby('proj_id')['gwl'].apply(lambda x: SD_diff(x))).reset_index()
sddiff_df = sddiff_df.rename(columns={'gwl':'flashiness'})
metrics_subset = metrics_subset.merge(sddiff_df[['proj_id', 'flashiness']], on='proj_id')

In [18]:
sddiff_nse_corr = create_corr_tbl(metrics_subset[metrics_subset['horizon']==12], TS_FEATURE='flashiness')
sddiff_nse_corr.to_excel(os.path.join(SHARE_PATH, 'global_mod_paper', 'results', 'nse_sddiff_spearman.xlsx'))
sddiff_nse_corr

,model_type,corr,p_val
0,nhits_dyn,-0.189498,0.0
1,nhits_full,-0.209856,0.0
2,tft_dyn,-0.225679,0.0
3,tft_full,-0.220185,0.0


#### Skewness

Boundedness, inhomogeneities, outliers, asymmetry of the probability distribution. 

$Skewness = \frac{mean-mode}{sd}$

- Zero and around zero: Time series with equal number of large an small amplitude values
- Positive: Many small values and few large values (right tail, left skewed)
- Negative: Many large values and few small values (left tail, right skewed)

In [19]:
skew_df = pd.DataFrame(compl_df.groupby('proj_id')['gwl'].skew()).reset_index()
skew_df = skew_df.rename(columns={'gwl':'skew'})

In [20]:
metrics_subset = metrics_subset.merge(skew_df[['proj_id', 'skew']], on='proj_id')
skew_nse_corr = create_corr_tbl(metrics_subset[metrics_subset['horizon']==12], TS_FEATURE='skew')
skew_nse_corr.to_excel(os.path.join(SHARE_PATH, 'global_mod_paper', 'results', 'nse_skew_spearman.xlsx'))
skew_nse_corr

,model_type,corr,p_val
0,nhits_dyn,-0.110151,1.793631e-170
1,nhits_full,-0.112875,5.826834e-179
2,tft_dyn,-0.076727,1.816800e-83
3,tft_full,-0.093079,4.310266e-122


#### Amplitude & Range ratio 

Amplitde = range.

Calculated on original, unscaled, unnormalized hydrographs

In [21]:
ampl_df = pd.DataFrame(compl_df.groupby('proj_id')['gwl'].apply(lambda x: x.max() - x.min())).reset_index()
ampl_df = ampl_df.rename(columns={'gwl':'amplitude'})

In [22]:
metrics_subset = metrics_subset.merge(ampl_df[['proj_id', 'amplitude']], on='proj_id')
ampl_nse_corr = create_corr_tbl(metrics_subset[metrics_subset['horizon']==12], TS_FEATURE='amplitude')
ampl_nse_corr.to_excel(os.path.join(SHARE_PATH, 'global_mod_paper', 'results', 'nse_ampl_spearman.xlsx'))
ampl_nse_corr

,model_type,corr,p_val
0,nhits_dyn,-0.116430,2.355555e-190
1,nhits_full,-0.126264,9.056979e-224
2,tft_dyn,-0.097563,5.332027e-134
3,tft_full,-0.115194,2.360592e-186


In [23]:
rr_df = pd.DataFrame(compl_df.groupby(['proj_id','year'])['gwl'].apply(lambda x: x.max() - x.min())).reset_index()
rr_df = rr_df.groupby(['proj_id']).agg(mean_range_gwl=('gwl', 'mean')).reset_index()
rr_df = rr_df.merge(ampl_df, on='proj_id')
rr_df.loc[:,'range_ratio'] = rr_df['mean_range_gwl'] / rr_df['amplitude'] 

In [24]:
metrics_subset = metrics_subset.merge(rr_df[['proj_id', 'range_ratio']], on='proj_id')
rr_nse_corr = create_corr_tbl(metrics_subset[metrics_subset['horizon']==12], TS_FEATURE='range_ratio')
rr_nse_corr.to_excel(os.path.join(SHARE_PATH, 'global_mod_paper', 'results', 'nse_rr_spearman.xlsx'))
rr_nse_corr

,model_type,corr,p_val
0,nhits_dyn,0.214353,0.000000e+00
1,nhits_full,0.173683,0.000000e+00
2,tft_dyn,0.088922,1.463845e-111
3,tft_full,0.097287,3.002649e-133


#### Trend

- Fit linear regression to the TS data
- Check if Coefficient is sig. positive or negative

In [25]:
def lin_trend(data: pd.DataFrame):
    _x = data['time_idx']
    _x = _x - _x.min()
    _y = data['gwl']
    
    slope, intercept, r, p, se = stats.linregress(_x, _y)
    return pd.Series({
        'slope':slope, 
        'p_value': p
    })

In [26]:
trend_df = compl_df.groupby('proj_id').apply(lin_trend).reset_index()

# If p above 0.05 set to zero
# Then correlate with slope
trend_df.loc[:,'slope_corrected'] = np.where(trend_df['p_value']>0.05, 0, trend_df['slope'])

In [27]:
metrics_subset = metrics_subset.merge(trend_df[['proj_id', 'slope_corrected']], on='proj_id')
slope_nse_corr = create_corr_tbl(metrics_subset[metrics_subset['horizon']==12], TS_FEATURE='slope_corrected')
slope_nse_corr.to_excel(os.path.join(SHARE_PATH, 'global_mod_paper', 'results', 'nse_slope_spearman.xlsx'))
slope_nse_corr

,model_type,corr,p_val
0,nhits_dyn,-0.022321,1.872842e-08
1,nhits_full,-0.020443,2.603313e-07
2,tft_dyn,0.031185,3.918352e-15
3,tft_full,0.009180,2.075418e-02


In [ ]:
# Differentiaded based on trend
metrics_subset.loc[:,'slope_cat'] = np.select(
    [
        metrics_subset['slope_corrected']==0, 
        metrics_subset['slope_corrected']<0,
        metrics_subset['slope_corrected']>0
    ], 
    [
        'No trend', 
        'Negative',
        'Positive'
    ]
)
metrics_subset[metrics_subset['horizon']==12].groupby(['model_type', 'slope_cat'])['NSE'].median()